# Практика
Используемые библиотеки

In [5]:
import pandas as pd
import numpy as np

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [6]:
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [7]:
# df.rename(columns={'Показатель': 'Indicator','Единица измерений': 'Unit','Результат анализа': 'Result','Норматив': 'Norma'}, inplace=True)
df['Результат анализа'] = df['Результат анализа'].replace('б/цвета', '0')
df['Норматив'] = df['Норматив'].str.replace(',', '.')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,0,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,не более 3.5
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [15]:
re.sub(r"[^\d\.]+","", "не более 0.3")

'0.3'

In [19]:
import re

def math_norm(row):
    value, norm = float(row[0]), row[1]
    if "не более" in norm:
        value_norm = float(re.sub(r"[^\d\.]+","", norm))
        return (value <= value_norm) * "В норме" + (value > value_norm) * "Выше норме"
    if "в пределах" in norm:
        norm_min, norm_max = re.sub(r"[^\d\-]+","", norm).split("-")
        norm_min, norm_max = float(norm_min), float(norm_max)
        return ((norm_min <= value) and (norm_max >= value) ) * "В норме" +   \
                (norm_min > value) * "Ниже нормы" +  (norm_max < value)  * "Выше нормы"


In [20]:
df["Вывод"] = df[["Результат анализа", "Норматив"]].apply(math_norm, axis=1)

/var/folders/yy/h36sr_t505d86h9cjh8p7lym0000gn/T/ipykernel_45064/3816984951.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value, norm = float(row[0]), row[1]


In [21]:
df

,Показатель,Единица измерений,Результат анализа,Норматив,Вывод
0,pH,единицы pH,8.4,в пределах 6-9,В норме
1,Запах,баллы,1,не более 2-3,В норме
2,Цветность,градусы,0,не более 30,В норме
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,В норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,В норме
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,В норме
6,Нитраты (по NO3),мг/дм3,24,не более 45,В норме
7,Фосфаты (P),мг/дм3,0.36,не более 3.5,В норме
8,Хлориды (Cl),мг/дм3,200,не более 350,В норме
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,В норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [27]:
import random
import numpy as nm
# заполняем ящик
box = ["orange"]*5 + ['apple']*4

# задаем число экспериментов
n = 1000000

# выборка
k = 3

# считаем вероятность
positive = []
for _ in range(n):
    random.shuffle(box)
    positive.append(all(x == 'orange' for x in box[:k]))
print(f'Эксперементальная вероятность {np.array(positive).mean()}')

Эксперементальная вероятность 0.118896


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [28]:
details = ['not_std']*3 + ["std"]*7

count_exp = 1000000

# выборка
k = 2

positive = []
for _ in range(count_exp):
    random.shuffle(details)
    cond_1 = all(x == 'not_std' for x in details[0:k-1])
    cond_2 = all(x == 'std' for x in details[k-1:k])
    positive.append(all([cond_1, cond_2]))
print(f'Эксперементальная вероятность {np.array(positive).mean()}')

Эксперементальная вероятность 0.233333
